# Wikipedia search

In [97]:
import wikipedia
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import math

# Get all the data from the wikipedia

In [131]:
# Infrmation of the book
author = "Keller, Helen"
title = "die geschichte meines lebens"
title_clean = "geschichte meines lebens" 
old_date = 1880 
birth = 1895 
death = 1975

author = author.replace(",", "")

# Search for the possible wiki-websites about the author
search = wikipedia.search(author)
search += wikipedia.search(title)
print(search)

['Helen Keller', 'Helen Keller Day', 'Helen Keller (disambiguation)', 'Helen Keller International', 'Helen Keller! The Musical', 'Helen Keller in Her Story', 'Helen Keller School', 'Statue of Helen Keller', 'Anne Sullivan', 'The Story of My Life (biography)', 'The Story of My Life (musical)', 'Georg Ebers', 'Adolph Freiherr Knigge', 'Karl Hase', 'Wachet auf, ruft uns die Stimme', 'List of compositions by Johann Sebastian Bach', 'Anna of Saxony (1567–1613)', 'List of German television series', 'Max Halbe', 'Hofmeister (office)']


In [147]:
# list to store the found sentences with the title
promts = []
# Function to get the html of the author, for easier usage in the for loop
def gethtml(author, lang = "de"):
    # replace the spaces with the _ so it can be used in the url
    author.replace(" ", "_")
    response = None 
    response = requests.get(url="https://" + lang + ".wikipedia.org/wiki/" + author)
    # get the content of the website
    soup = BeautifulSoup(response.content, 'html.parser')
    content = soup.find(id="content").get_text()
    return content.__str__()

# finds all parts of the book in the text, and returns (yields) them one by one
def find_all_parts(content, word):
    n = 0
    while True:
        n = content.find(word, n)
        if n == -1:
            break
        yield n
        n += len(word)

# only select the row arround the found word
def select_row(ip):
    enters = []
    for i in range(0, len(promts[ip])):
       c = promts[ip][i]
       if c == '\n':
           enters.append(i)

    promt_new = ''
    if len(enters) > 1:
       first_index = min(enters, key=lambda x:abs(x-margin))
       if first_index <= 250:
           if enters.index(first_index) < (len(enters) - 1):
               promt_new = promts[ip][first_index:enters[enters.index(first_index) + 1]]
           else:
               promt_new = promts[ip][first_index:]
       elif first_index >= 250:
           if enters.index(first_index) > 0:
               promt_new = promts[ip][enters[enters.index(first_index) - 1]:first_index]
           else:
               promt_new = promts[ip][:first_index]

    if promt_new != '':
        promts[ip] = promt_new

def find_title(content, word):
    for w in word.split(" "):
        pattern = re.compile(r'{}'.format(w), re.IGNORECASE)
        match = pattern.search(content)
        if match:
            return [match.start(), match.end()]
        return [None, None]

def find_dates(text: str):
    title_start, title_end = find_title(text, title_clean)

    years = pd.DataFrame(columns=['word', 'start', 'end', 'distance'])
    n = 0
    year_start, year_end = 0, 0
    year = ""
    for i in range(0, len(text) + 1):
        if i < len(text):
            c = text[i]
            if c.isnumeric():
                if n == 0:
                    year_start = i
                n += 1
                year += c
                continue
        if n != 0 and n <= 4:
            year_end = i
            try:
                year = int(year)
                if year < birth or year > death:
                    n = 0
                    year = ""
                    continue

                dis1, dis2, dis3, dis4 = year_start - title_start, year_start - title_end, year_end - title_start, year_end - title_end
                dis = min(abs(dis1), abs(dis2), abs(dis3), abs(dis4))

                years = years._append({'word': year, 'start': year_start, 'end': year_end, 'distance': dis}, ignore_index=True)
                if dis == 1:
                    print(repr(promts[i]))
            except:
                print("could not parse")
        n = 0
        year = ""
    return years


df = pd.DataFrame(columns=['word', 'start', 'end', 'distance'])
responses = []
length = 0

# goes through the content of the given pages and saves only the text surrounding the found title parts
def find_promt(content, margin, length):
    for p in title_clean.split(" "):
        has_title = False
        for gen in find_all_parts(content, p):
            n = gen
            if n == -1:
                continue
            m = min(places, key=lambda x:abs(x-n))
            if m < margin and m > -margin:
                continue
            places.append(n)
            # get the sentence and text around the title
            if n < margin:
                text_to_check = content[:n+margin]
            else:
                text_to_check = content[n-margin:n+margin]
            info = "Search: " + s + "; Word in Text: " + p
            info += " (" + str(n) + "):\n"
            # info += text_to_check + "\n"
            print(info)
            # add the sentence to the list
            promts.append(text_to_check)
            print(text_to_check)
            select_row(len(promts)-1)


            promt = promts[len(promts) - 1]
            title_start, title_end = find_title(promt, title_clean)
            print(":", title_start, title_end)

            responses.append(find_dates(promt))
            response = responses[len(responses)-1]
            print(response)

            length += len(responses)
            if title_start == None or title_end == None:
                continue
            for j in range(0,len(response)):
                df.loc[df.count()['word']] = response.iloc[j]

# how much text should be selected arround a found word
margin = 250
for s in search:
    # look for the position of the title in the content
    places = []
    places.append(-margin*margin)
    lang = "de"
    content = gethtml(s, lang)
    find_promt(content, margin, length)
    lang = "en"
    content = gethtml(s, lang)
    find_promt(content, margin, length)


print(len(promts))

Search: Helen Keller; Word in Text: geschichte (12602):

iography. Knopf, New York 1998, ISBN 0-679-44354-1.
Joseph P. Lash: Helen and teacher. The Story of Helen Keller and Anne Sullivan Macy. Delacorte Press, New York NY 1980, ISBN 0-440-03654-2.
Katja Behrens: Alles Sehen kommt von der Seele. Die Lebensgeschichte der Helen Keller. Beltz & Gelberg, Weinheim u. a. 2001, ISBN 3-407-80889-5.
Helen E. Waite: Öffne mir das Tor zur Welt. 4., durchgesehene und verbesserte  Auflage. Freies Geistesleben, Stuttgart 1999, ISBN 3-7725-1850-8. 
Werner Pieper: Bl
: 59 69
Empty DataFrame
Columns: [word, start, end, distance]
Index: []
Search: Helen Keller; Word in Text: meines (11013):

Schulen, auch im deutschsprachigen Raum, sind nach Helen Keller benannt worden. In Hannover-Kirchrode wurde im Jahr 2000 ein Fußweg in der Nähe des Deutschen Taubblindenwerks nach ihr benannt.

Werke[Bearbeiten | Quelltext bearbeiten]
Die Geschichte meines Lebens. Robert Lutz, Stuttgart 1905, (deutsche Übersetzung d

# Searches for the year in the found sentences

In [134]:
def find_title(content, word):
    for w in word.split(" "):
        pattern = re.compile(r'{}'.format(w), re.IGNORECASE)
        match = pattern.search(content)
        if match:
            return [match.start(), match.end()]
        return [None, None]

def find_dates(text: str):
    title_start, title_end = find_title(text, title_clean)

    years = pd.DataFrame(columns=['word', 'start', 'end', 'distance'])
    n = 0
    year_start, year_end = 0, 0
    year = ""
    for i in range(0, len(text) + 1):
        if i < len(text):
            c = text[i]
            if c.isnumeric():
                if n == 0:
                    year_start = i
                n += 1
                year += c
                continue
        if n != 0 and n <= 4:
            year_end = i
            try:
                year = int(year)

                dis1, dis2, dis3, dis4 = year_start - title_start, year_start - title_end, year_end - title_start, year_end - title_end
                dis = min(abs(dis1), abs(dis2), abs(dis3), abs(dis4))

                years = years._append({'word': year, 'start': year_start, 'end': year_end, 'distance': dis}, ignore_index=True)
                if dis == 1:
                    print(repr(promts[i]))
            except:
                print("could not parse")
        n = 0
        year = ""
    return years


df = pd.DataFrame(columns=['word', 'start', 'end', 'distance'])
responses = []
length = 0
for i in range(len(promts)):
    title_start, title_end = find_title(promts[i], title_clean)
    print(str(i) + ":", title_start, title_end)

    responses.append(find_dates(promts[i]))
    print(responses[i])

    length += len(responses)
    if title_start == None or title_end == None:
        continue
    for j in range(0,len(responses[i])):
        df.loc[df.count()['word']] = responses[i].iloc[0]

print(length)

0: 59 69
   word start  end distance
0  2001   120  124       51
1     3   131  132       62
2   407   133  136       64
3     5   143  144       74
1: 5 15
   word start  end distance
0  1905    54   58       39
1  1905   187  191      172
2: 218 228
   word start  end distance
0  1919     7   11      207
1  1962   318  322       90
3: 93 103
   word start  end distance
0  2014    18   22       71
1    10   240  242      137
4: 8 18
   word start end distance
0  2014    39  43       21
1  2023    86  90       68
2     5    91  93       73
3     3    94  96       76
5: 8 18
   word start  end distance
0  2016    87   91       69
1     4   114  115       96
2  2024   122  126      104
6: 8 18
   word start  end distance
0  2016    87   91       69
1     4   114  115       96
2  2024   122  126      104
7: 5 15
   word start end distance
0  1893    65  69       50
8: 24 34
   word start end distance
0  1893    84  88       50
9: None None
could not parse
could not parse
could not parse
E

In [145]:
print(df)

    word start  end  distance
0   1905    54   58        39
1   1905   187  191       172
2   1919     7   11       207
3   1962   318  322        90
4   1895    67   71        82
5   1900   329  333        69
6   1933   268  272       137
7   1914   325  329       194
8   1935   343  347       212
9   1933   410  414       279
10  1933   212  216       137
11  1914   269  273       194
12  1935   287  291       212
13  1933   354  358       279
14  1933    50   54        16
15  1914    49   53        20
16  1935    62   66        33
17  1945   102  106        73
18  1933    65   69        31
19  1914    49   53        20
20  1935    70   74        41


# Analyse the data with the whole set, without counting the occurances

In [146]:
df['distance'] = pd.to_numeric(df['distance'])
print(df.sort_values(by='distance')[0:10])

df_winners = df.sort_values(by='distance')[0:10].groupby('word')['distance'].mean().reset_index()

# Rename columns for clarity
df_winners.columns = ['year', 'average_distance']
df_winners = pd.DataFrame(df_winners)

df_year_counts = df.sort_values(by='distance')[0:10].groupby('word').count().reset_index()[['word', 'distance']]
df_year_counts.columns = ['year', 'count']

df_winners = pd.merge(df_winners, df_year_counts, on='year')

df_winners

calcs = {}
for i in df_winners.iterrows():
    # 1915: 0.99 (score) + 0.13 (count) = 1.12
    calc_count = (i[1]['count'] - df_winners.min()['count'])/(df_winners.max()['count'] - df_winners.min()['count'])
    calc_dist = 1 - (i[1]['average_distance'] - df_winners.min()['average_distance'])/(df_winners.max()['average_distance'] - df_winners.min()['average_distance'])
    if math.isnan(calc_count): calc_count = 1
    if math.isnan(calc_dist): calc_dist = 1
    calc = (calc_dist + calc_count)/2
    print(i[1]['year'],  calc_dist, '(dist' , str(round(i[1]['average_distance'])) + ')', calc_count, '(count', str(round(i[1]['count'])) + ') =', calc)
    calcs[i[1]['year']] = calc
print()
win = max(calcs, key=calcs.get)
print(win, calcs[win])

dict(sorted(calcs.items(), key=lambda item: item[1]))

    word start  end  distance
14  1933    50   54        16
15  1914    49   53        20
19  1914    49   53        20
18  1933    65   69        31
16  1935    62   66        33
0   1905    54   58        39
20  1935    70   74        41
5   1900   329  333        69
17  1945   102  106        73
4   1895    67   71        82
1895.0 0.0 (dist 82) 0.0 (count 1) = 0.0
1900.0 0.20967741935483875 (dist 69) 0.0 (count 1) = 0.10483870967741937
1905.0 0.6935483870967742 (dist 39) 0.0 (count 1) = 0.3467741935483871
1914.0 1.0 (dist 20) 1.0 (count 2) = 1.0
1933.0 0.9435483870967742 (dist 24) 1.0 (count 2) = 0.9717741935483871
1935.0 0.7258064516129032 (dist 37) 1.0 (count 2) = 0.8629032258064516
1945.0 0.14516129032258063 (dist 73) 0.0 (count 1) = 0.07258064516129031

1914.0 1.0


{1895.0: 0.0,
 1945.0: 0.07258064516129031,
 1900.0: 0.10483870967741937,
 1905.0: 0.3467741935483871,
 1935.0: 0.8629032258064516,
 1933.0: 0.9717741935483871,
 1914.0: 1.0}